In [ ]:
import hashlib # 这里使用了Python标准库中的hashlib模块来计算SHA2-256哈希，仅仅是为了简化代码，实际实现已经放在HMAC_SHA2_256.py中，可进行查看

def expand_key(key_hex: str) -> str:
    """扩展密钥到64字节的整数倍"""
    key_bytes = bytes.fromhex(key_hex)
    if len(key_bytes) > 64:
        key_bytes = bytes.fromhex(hashlib.sha256(key_bytes).hexdigest())
    target_length = ((len(key_bytes) - 1) // 64 + 1) * 64
    expanded_bytes = key_bytes.ljust(target_length, b'\x00')
    return expanded_bytes.hex()

def xor_with_ipad(expanded_key_hex: str) -> str:
    """与ipad (0x36) 进行异或操作"""
    expanded_key_bytes = bytes.fromhex(expanded_key_hex)
    ipad_bytes = bytes([0x36] * len(expanded_key_bytes))
    xor_result = bytes(a ^ b for a, b in zip(expanded_key_bytes, ipad_bytes))
    return xor_result.hex()

def xor_with_opad(expanded_key_hex: str) -> str:
    """与opad (0x5C) 进行异或操作"""
    expanded_key_bytes = bytes.fromhex(expanded_key_hex)
    opad_bytes = bytes([0x5C] * len(expanded_key_bytes))
    xor_result = bytes(a ^ b for a, b in zip(expanded_key_bytes, opad_bytes))
    return xor_result.hex()

def sha256_hex(data_hex: str) -> str:
    """计算SHA2-256哈希并返回十六进制字符串"""
    data_bytes = bytes.fromhex(data_hex)
    return hashlib.sha256(data_bytes).hexdigest()

def hmac_sha256(key_hex: str, message_hex: str) -> str:
    """使用SHA2-256计算HMAC"""
    # 步骤1: 扩展密钥
    expanded_key = expand_key(key_hex)
    print(f"Expanded Key: {expanded_key}")
    
    # 步骤2: 计算K0^ipad
    k0_xor_ipad = xor_with_ipad(expanded_key)
    print(f"K0^ipad: {k0_xor_ipad}")
    
    # 步骤3: 计算SHA2-256((K0^ipad)||message)
    inner_data = k0_xor_ipad + message_hex
    inner_hash = sha256_hex(inner_data)
    print(f"Inner Hash: {inner_hash}")
    
    # 步骤4: 计算K0^opad
    k0_xor_opad = xor_with_opad(expanded_key)
    print(f"K0^opad: {k0_xor_opad}")
    
    # 步骤5: 计算最终MAC
    outer_data = k0_xor_opad + inner_hash
    mac = sha256_hex(outer_data)
    
    return mac

In [ ]:
# 本代码仅用作校验正确性
import hashlib
import hmac

def hmac_sha256_std(key_hex: str, message_hex: str) -> str:
    """使用Python内置的hmac库计算HMAC-SHA256"""
    key_bytes = bytes.fromhex(key_hex)
    # print(f"Key Bytes: {key_bytes.hex()}")
    message_bytes = bytes.fromhex(message_hex)
    # print(f"Message Bytes: {message_bytes.hex()}")
    hmac_result = hmac.new(key_bytes, message_bytes, hashlib.sha256).hexdigest().upper()
    return hmac_result

In [ ]:
key = "000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F202122232425262728292A2B2C2D2E2F303132333435363738393A3B3C3D3E3F"
message = "53616D706C65206D65737361676520666F72206B65796C656E3D626C6F636B6C656E"  
hmac_sha256_result = hmac_sha256(key, message).upper()
print(f"HMAC-SHA256: {hmac_sha256_result}") 

expected_mac = hmac_sha256_std(key, message).upper()
print(f"Expected MAC: {expected_mac}")

if hmac_sha256_result == expected_mac:
    print("HMAC-SHA256 计算正确！")
else:
    print("HMAC-SHA256 计算错误！")

print("使用标准库计算的HMAC-SHA256:", hmac_sha256_std(key, message).upper())

Expanded Key: 000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f
K0^ipad: 36373435323330313e3f3c3d3a3b383926272425222320212e2f2c2d2a2b282916171415121310111e1f1c1d1a1b181906070405020300010e0f0c0d0a0b0809
Inner Hash: c0918e14c43562b910db4b8101cf8812c3da2783c670bff34d88b3b88e731716
K0^opad: 5c5d5e5f58595a5b54555657505152534c4d4e4f48494a4b44454647404142437c7d7e7f78797a7b74757677707172736c6d6e6f68696a6b6465666760616263
HMAC-SHA256: 8BB9A1DB9806F20DF7F77B82138C7914D174D59E13DC4D0169C9057B133E1D62
Expected MAC: 8BB9A1DB9806F20DF7F77B82138C7914D174D59E13DC4D0169C9057B133E1D62
HMAC-SHA256 计算正确！
使用标准库计算的HMAC-SHA256: 8BB9A1DB9806F20DF7F77B82138C7914D174D59E13DC4D0169C9057B133E1D62


In [ ]:
key = "000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F"
message = "53616D706C65206D65737361676520666F72206B65796C656E3D626C6F636B6C656E"  
hmac_sha256_result = hmac_sha256(key, message).upper()
print(f"HMAC-SHA256: {hmac_sha256_result}") 

expected_mac = hmac_sha256_std(key, message).upper()
print(f"Expected MAC: {expected_mac}")

if hmac_sha256_result == expected_mac:
    print("HMAC-SHA256 计算正确！")
else:
    print("HMAC-SHA256 计算错误！")

print("使用标准库计算的HMAC-SHA256:", hmac_sha256_std(key, message).upper())

Expanded Key: 000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f0000000000000000000000000000000000000000000000000000000000000000
K0^ipad: 36373435323330313e3f3c3d3a3b383926272425222320212e2f2c2d2a2b28293636363636363636363636363636363636363636363636363636363636363636
Inner Hash: 7c1a4aae7d2454888f28094528a3411b2f92df65865988338160a45f1acdc863
K0^opad: 5c5d5e5f58595a5b54555657505152534c4d4e4f48494a4b44454647404142435c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c
HMAC-SHA256: 648C89DC60D3D2EE50B5A2D116FDB7583EB98DC1AA90AAB3DFF3ECFD02AC90BE
Expected MAC: 648C89DC60D3D2EE50B5A2D116FDB7583EB98DC1AA90AAB3DFF3ECFD02AC90BE
HMAC-SHA256 计算正确！
使用标准库计算的HMAC-SHA256: 648C89DC60D3D2EE50B5A2D116FDB7583EB98DC1AA90AAB3DFF3ECFD02AC90BE


In [ ]:
key = "000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F202122232425262728292A2B2C2D2E2F303132333435363738393A3B3C3D3E3F404142434445464748494A4B4C4D4E4F505152535455565758595A5B5C5D5E5F60616263"
message = "53616D706C65206D65737361676520666F72206B65796C656E3D626C6F636B6C656E"
hmac_sha256_result = hmac_sha256(key, message).upper()
print(f"HMAC-SHA256: {hmac_sha256_result}")

expected_mac = hmac_sha256_std(key, message).upper()
print(f"Expected MAC: {expected_mac}")

if hmac_sha256_result == expected_mac:
    print("HMAC-SHA256 计算正确！")
else:
    print("HMAC-SHA256 计算错误！")

print("使用标准库计算的HMAC-SHA256:", hmac_sha256_std(key, message).upper())

Expanded Key: bce0aff19cf5aa6a7469a30d61d04e4376e4bbf6381052ee9e7f33925c954d520000000000000000000000000000000000000000000000000000000000000000
K0^ipad: 8ad699c7aac39c5c425f953b57e6787540d28dc00e2664d8a84905a46aa37b643636363636363636363636363636363636363636363636363636363636363636
Inner Hash: 1e0dfb0cbb61e9f060769e9df57501292426f0db58194bc85bc63dac4670c2c1
K0^opad: e0bcf3adc0a9f6362835ff513d8c121f2ab8e7aa644c0eb2c2236fce00c9110e5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c
HMAC-SHA256: BDCCB6C72DDEADB500AE768386CB38CC41C63DBB0878DDB9C7A38A431B78378D
Expected MAC: BDCCB6C72DDEADB500AE768386CB38CC41C63DBB0878DDB9C7A38A431B78378D
HMAC-SHA256 计算正确！
使用标准库计算的HMAC-SHA256: BDCCB6C72DDEADB500AE768386CB38CC41C63DBB0878DDB9C7A38A431B78378D


In [ ]:
key = "000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F202122232425262728292A2B2C2D2E2F30"
message = "53616D706C65206D65737361676520666F72206B65796C656E3C626C6F636B6C656E2C2077697468207472756E636174656420746167"
hmac_sha256_result = hmac_sha256(key, message).upper()
print(f"HMAC-SHA256: {hmac_sha256_result}")

expected_mac = hmac_sha256_std(key, message).upper()
print(f"Expected MAC: {expected_mac}")

if hmac_sha256_result == expected_mac:
    print("HMAC-SHA256 计算正确！")
else:
    print("HMAC-SHA256 计算错误！")

print("使用标准库计算的HMAC-SHA256:", hmac_sha256_std(key, message).upper())

Expanded Key: 000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f30000000000000000000000000000000
K0^ipad: 36373435323330313e3f3c3d3a3b383926272425222320212e2f2c2d2a2b282916171415121310111e1f1c1d1a1b181906363636363636363636363636363636
Inner Hash: cba02e369d29352fbae861944b264187a7d8c1d22cdaf9f5d746556cfe74ddbe
K0^opad: 5c5d5e5f58595a5b54555657505152534c4d4e4f48494a4b44454647404142437c7d7e7f78797a7b74757677707172736c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c
HMAC-SHA256: 27A8B157839EFEAC98DF070B331D593618DDB985D403C0C786D23B5D132E57C7
Expected MAC: 27A8B157839EFEAC98DF070B331D593618DDB985D403C0C786D23B5D132E57C7
HMAC-SHA256 计算正确！
使用标准库计算的HMAC-SHA256: 27A8B157839EFEAC98DF070B331D593618DDB985D403C0C786D23B5D132E57C7
